In [1]:
%matplotlib inline
import numpy as np
import sklearn
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
import timeit
import pandas as pd
import math
import itertools
import time

import sklearn.neighbors
from sklearn.neighbors import (
    KernelDensity,
    KDTree,
)
from sklearn.preprocessing import (
    RobustScaler
)
import sklearn.decomposition
import sklearn.covariance
import sklearn.mixture
import sklearn.svm

In [2]:
df = pd.read_csv("../data/home_sensor.csv")

In [14]:
values = df[list(range(3,7))].head(500000)

In [18]:
def estimate_kde_bw(data):
    q3 = np.percentile(data, 75, axis=0)
    q1 = np.percentile(data, 25, axis=0)
    iqr = q3 - q1
    bw = iqr * (data.shape[0])**(-1.0/(data.shape[1]+4))
    return bw

In [19]:
bw = estimate_kde_bw(values)
norm_data = values / bw

# GMM

In [10]:
def calc_gmm(k, data):
    gmm = sklearn.mixture.GaussianMixture(
        n_components=k,
        covariance_type="diag",
        random_state=0,
    )
    gmm.fit(data)
    gmm_scores = gmm.score_samples(data)
    return gmm_scores

In [15]:
t = time.time()
calc_gmm(10,values)
print(time.time() - t)

17.687546968460083


# OCSVM

In [17]:
def calc_svm(nu, gamma, data):
    svm = sklearn.svm.OneClassSVM(nu=nu,kernel="rbf",gamma=gamma)
    svm.fit(data)
    scores = svm.decision_function(data)

In [22]:
t = time.time()
calc_svm(0.01, 1, norm_data.iloc[:100000])